In [1]:
import warnings; warnings.filterwarnings(action='ignore')
%matplotlib inline
#for Netcdf manipulation
import xarray as xr
from netCDF4 import Dataset
import netCDF4 as nc

#for array manipulation
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Grouper

In [2]:
#Reading files with xarray
sia_file = '/Users/fridaperez/Developer/repos/local_repo/Area_SH_Filled-316x332.nc'  # prepping path for xr
sit_file = '/Users/fridaperez/Developer/repos/local_repo/SIT_25000/SIT_SH_Filled-316x332.nc'
#sitlatlon= '/Users/fridaperez/Developer/repos/volume-thickness/SIT_files/SIT_CYS_25k_interp3.nc'

data_sia = xr.open_dataset(sia_file) # opening sit for xarray
data_sit= xr.open_dataset(sit_file)
#datall = xr.open_dataset(sitlatlon)

In [4]:
data_sit

<xarray.Dataset>
Dimensions:    (time: 97, xgrid: 316, ygrid: 332)
Coordinates:
    longitude  (ygrid, xgrid) float64 ...
    latitude   (ygrid, xgrid) float64 ...
  * ygrid      (ygrid) float32 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
  * xgrid      (xgrid) float32 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * time       (time) datetime64[ns] 2002-10-15 2003-05-15 ... 2018-10-15
Data variables:
    SIT        (time, ygrid, xgrid) float64 ...
Attributes:
    history:  Thu May 19 22:02:49 2022: ncpdq -a time,ygrid,xgrid SIT_SH_316x...
    NCO:      netCDF Operators version 4.9.6 (Homepage = http://nco.sf.net, C...

In [5]:
latitude = data_sit.latitude
longitude = data_sit.longitude
time = data_sit["time"]

In [7]:
#converting xarray.dataArray to numpy array
nu_sia = data_sia['__xarray_dataarray_variable__'].values
nu_sit = data_sit['SIT'].values

In [8]:
nu_sit.shape

(97, 332, 316)

In [9]:
prod = np.multiply(nu_sia, nu_sit)
prod.shape

(97, 332, 316)

In [10]:
volume = prod

In [11]:
volume.shape

(97, 332, 316)

In [12]:
da = xr.DataArray(
    data=volume,
    dims=["time","ygrid","xgrid"],
    coords=dict(
        time=time,
        longitude=(["ygrid","xgrid"],longitude),
        latitude=(["ygrid","xgrid"],latitude),
    ),
    attrs=dict(
        description="Sea Ice Volume.",
        units="km3",
    ),
)

In [13]:
da.to_netcdf('/Users/fridaperez/Developer/repos/local_repo/Merged_Volume_2002-2018.nc')

In [14]:
file = '/Users/fridaperez/Developer/repos/local_repo/Merged_Volume_2002-2018.nc'
datos = xr.open_dataset(file)

In [15]:
newset=datos.rename_vars({'__xarray_dataarray_variable__':'volume'})

In [16]:
newset.to_netcdf('/Users/fridaperez/Developer/repos/local_repo/Volume_SH_ENV-CY2_2002-2018.nc')